In [1]:
import pandas as pd
Tenders = pd.read_excel("C:/Users/Mitch/git/UWACapstoneG2/data/UpdatedTenders.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Mitch/git/UWACapstoneG2/data/UpdatedTenders.xlsx'

In [ ]:
# unique links
Prep = Tenders["TenderLink"].unique()
# without nan
Links = Prep[~pd.isnull(Prep)]

In [ ]:
# Extracts tender zip files from links
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# buttons 
locators = {
    "Download Now": By.LINK_TEXT,
    "Download for Information Only": By.XPATH,
    "Download Documents": By.XPATH
}

custom_user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36"

def click_button(driver, wait, button_text):
    
    locator = locators[button_text]
    if locator == "xpath":
        button_text = f"//input[@value='{button_text}']"
    
    try:
        button = wait.until(EC.element_to_be_clickable((locator, button_text)))
        button.click()
    except Exception as e:
        print(f"Error while clicking button '{button_text}': {e}")

def get_tenders(link):
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument(f"user-agent={custom_user_agent}")
    driver = webdriver.Chrome(options = options)

    driver.get(link)

    # wait for button to be clickable
    wait = WebDriverWait(driver, 10)
    
    buttons = ["Download Now", "Download for Information Only", "Download Documents"]
    for button in buttons:
        click_button(driver, wait, button)

    # enough time to download files before quitting
    time.sleep(5)
    driver.quit()

In [ ]:
# get num workers, how many files to download at the same time
num_threads = 10 
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    executor.map(get_tenders, Links)

In [ ]:
get_tenders(Links[0])